# Chapter 2: Simulation Environments

This notebook explores various simulation environments for robotics, including Gazebo, Unity, and custom physics simulators. We'll learn how to create simulation scenarios and test robotics algorithms in a safe environment.

In [ ]:
# Configuration cell - select execution mode
EXECUTION_MODE = "simulation"  # Options: "hardware", "simulation"

print(f"Simulation Chapter 2: Running in {EXECUTION_MODE} mode")
print("Initializing simulation environment...")

# Import required libraries
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import matplotlib.animation as animation

print("Simulation environment setup complete!")

## 1. Understanding Simulation Environments

Simulation environments allow us to test robotics algorithms in a safe, controlled environment before deploying to real hardware. We'll explore basic physics simulation concepts.

In [ ]:
# Create a simple 2D physics simulation environment
class SimplePhysicsSimulator:
    def __init__(self, width=10, height=10):
        self.width = width
        self.height = height
        self.objects = []
        print(f"Physics simulator created: {width}x{height} environment")
    
    def add_object(self, name, x, y, width=1, height=1, color='blue'):
        obj = {
            'name': name,
            'x': x,
            'y': y,
            'width': width,
            'height': height,
            'color': color,
            'vx': 0,  # velocity x
            'vy': 0   # velocity y
        }
        self.objects.append(obj)
        print(f"Added {name} at position ({x}, {y})")
        return obj
    
    def apply_force(self, obj, fx, fy):
        # Simple force application (F = ma, so a = F/m, assuming m=1)
        obj['vx'] += fx
        obj['vy'] += fy
        print(f"Applied force ({fx}, {fy}) to {obj['name']}")
    
    def update(self, dt=0.1):
        for obj in self.objects:
            # Update position based on velocity
            obj['x'] += obj['vx'] * dt
            obj['y'] += obj['vy'] * dt
            
            # Simple boundary collision
            if obj['x'] < 0:
                obj['x'] = 0
                obj['vx'] *= -0.8  # bounce with damping
            elif obj['x'] + obj['width'] > self.width:
                obj['x'] = self.width - obj['width']
                obj['vx'] *= -0.8
                
            if obj['y'] < 0:
                obj['y'] = 0
                obj['vy'] *= -0.8
            elif obj['y'] + obj['height'] > self.height:
                obj['y'] = self.height - obj['height']
                obj['vy'] *= -0.8
    
    def get_object_position(self, name):
        for obj in self.objects:
            if obj['name'] == name:
                return obj['x'], obj['y']
        return None
    
    def render(self):
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xlim(0, self.width)
        ax.set_ylim(0, self.height)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
        ax.set_title('Simple Physics Simulation Environment')
        
        # Draw the environment boundaries
        boundary = Rectangle((0, 0), self.width, self.height, 
                           linewidth=2, edgecolor='black', facecolor='lightgray', alpha=0.3)
        ax.add_patch(boundary)
        
        # Draw all objects
        for obj in self.objects:
            rect = Rectangle((obj['x'], obj['y']), obj['width'], obj['height'],
                           linewidth=1, edgecolor='black', facecolor=obj['color'], alpha=0.7)
            ax.add_patch(rect)
            # Add label
            ax.text(obj['x'] + obj['width']/2, obj['y'] + obj['height']/2, 
                   obj['name'], ha='center', va='center', color='white', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

# Create a simulation environment
simulator = SimplePhysicsSimulator(width=15, height=10)
print("Simple physics simulator initialized")

## 2. Creating Robot Models in Simulation

Let's create a simple robot model and place it in our simulation environment.

In [ ]:
# Add a robot to the simulation
robot = simulator.add_object("Robot", x=2, y=2, width=1.5, height=1, color='green')
print(f"Robot added at initial position: ({robot['x']}, {robot['y']})")

# Add some obstacles
obstacle1 = simulator.add_object("Obstacle1", x=8, y=5, width=1, height=2, color='red')
obstacle2 = simulator.add_object("Obstacle2", x=12, y=3, width=1.5, height=1.5, color='orange')

# Show the initial state
simulator.render()

## 3. Applying Forces and Movement

Now let's apply forces to our robot and see how it moves in the environment.

In [ ]:
# Apply forces to move the robot
print("Applying forces to robot...")
simulator.apply_force(robot, fx=5, fy=0)  # Move right

# Run simulation for a few steps
print("Running simulation for 10 steps...")
for step in range(10):
    print(f"Step {step + 1}: Robot at ({robot['x']:.2f}, {robot['y']:.2f}), velocity ({robot['vx']:.2f}, {robot['vy']:.2f})")
    simulator.update(dt=0.2)
    time.sleep(0.1)  # Brief pause for visibility

# Show the final state
print(f"\nFinal robot position: ({robot['x']:.2f}, {robot['y']:.2f})")
simulator.render()

## 4. Sensor Simulation

In real robotics, sensors provide crucial information about the environment. Let's simulate some basic sensors.

In [ ]:
class SimpleSensorSimulator:
    def __init__(self, simulator):
        self.simulator = simulator
        
    def simulate_lidar(self, robot_name, max_range=5.0, num_rays=8):
        robot_pos = self.simulator.get_object_position(robot_name)
        if robot_pos is None:
            return []
        
        rx, ry = robot_pos
        readings = []
        
        # Simulate rays in different directions
        for i in range(num_rays):
            angle = 2 * np.pi * i / num_rays
            
            # Check for obstacles in this direction
            min_distance = max_range
            for obj in self.simulator.objects:
                if obj['name'] == robot_name:  # Skip the robot itself
                    continue
                    
                # Simple distance check
                obj_center_x = obj['x'] + obj['width'] / 2
                obj_center_y = obj['y'] + obj['height'] / 2
                distance = np.sqrt((obj_center_x - rx)**2 + (obj_center_y - ry)**2)
                
                if distance < min_distance:
                    min_distance = distance
            
            readings.append(min_distance if min_distance < max_range else max_range)
        
        return readings
    
    def simulate_camera(self, robot_name, fov=60):
        # Simple camera simulation - return what objects are in front of the robot
        robot_obj = None
        for obj in self.simulator.objects:
            if obj['name'] == robot_name:
                robot_obj = obj
                break
        
        if robot_obj is None:
            return []
        
        # In a real sim we would check what's in the robot's field of view
        # For simplicity, return all objects within a certain distance
        robot_x = robot_obj['x'] + robot_obj['width'] / 2
        robot_y = robot_obj['y'] + robot_obj['height'] / 2
        
        visible_objects = []
        for obj in self.simulator.objects:
            if obj['name'] == robot_name:
                continue
                
            obj_x = obj['x'] + obj['width'] / 2
            obj_y = obj['y'] + obj['height'] / 2
            distance = np.sqrt((obj_x - robot_x)**2 + (obj_y - robot_y)**2)
            
            if distance < 5.0:  # Within 5 units
                visible_objects.append({
                    'name': obj['name'],
                    'distance': distance,
                    'direction': np.arctan2(obj_y - robot_y, obj_x - robot_x)
                })
        
        return visible_objects

# Create sensor simulator
sensor_sim = SimpleSensorSimulator(simulator)

# Test the sensors
lidar_readings = sensor_sim.simulate_lidar("Robot", num_rays=12)
camera_objects = sensor_sim.simulate_camera("Robot")

print(f"Lidar readings: {[f'{r:.2f}' for r in lidar_readings]}")
print(f"Camera objects: {camera_objects}")

## 5. Navigation and Path Planning

Let's implement a simple path planning algorithm in our simulation.

In [ ]:
def simple_path_planner(start, goal, obstacles, grid_resolution=0.5):
    """Simple path planning that avoids obstacles"""
    sx, sy = start
    gx, gy = goal
    
    # For simplicity, we'll create a path that goes around obstacles
    # In a real system, we'd use A* or RRT algorithms
    path = [start]
    
    # Simple approach: go to a point above all obstacles, then to goal
    max_y = max([obj['y'] + obj['height'] for obj in obstacles]) + 2
    
    # Add intermediate point above obstacles
    path.append((sx, max_y))
    path.append((gx, max_y))
    path.append(goal)
    
    return path

# Define start and goal positions
start_pos = (1, 1)
goal_pos = (13, 8)

# Extract obstacle information
obstacles = [obj for obj in simulator.objects if obj['name'] != 'Robot']

# Plan a path
path = simple_path_planner(start_pos, goal_pos, obstacles)
print(f"Planned path: {path}")

# Visualize the path
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(0, simulator.width)
ax.set_ylim(0, simulator.height)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_title('Path Planning in Simulation Environment')

# Draw the environment boundaries
boundary = Rectangle((0, 0), simulator.width, simulator.height, 
                   linewidth=2, edgecolor='black', facecolor='lightgray', alpha=0.3)
ax.add_patch(boundary)

# Draw all objects
for obj in simulator.objects:
    rect = Rectangle((obj['x'], obj['y']), obj['width'], obj['height'],
                   linewidth=1, edgecolor='black', facecolor=obj['color'], alpha=0.7)
    ax.add_patch(rect)
    ax.text(obj['x'] + obj['width']/2, obj['y'] + obj['height']/2, 
           obj['name'], ha='center', va='center', color='white', fontweight='bold')

# Draw the path
path_x = [p[0] for p in path]
path_y = [p[1] for p in path]
ax.plot(path_x, path_y, 'g--', linewidth=2, label='Planned Path', marker='o')

# Mark start and goal
ax.plot(start_pos[0], start_pos[1], 'go', markersize=10, label='Start', markeredgecolor='black')
ax.plot(goal_pos[0], goal_pos[1], 'ro', markersize=10, label='Goal', markeredgecolor='black')

ax.legend()
plt.tight_layout()
plt.show()

## 6. Simulation Control and Experimentation

Let's create a more sophisticated simulation that allows us to control the robot and test different scenarios.

In [ ]:
class AdvancedRobotController:
    def __init__(self, simulator, robot_name):
        self.simulator = simulator
        self.robot_name = robot_name
        self.robot = None
        for obj in simulator.objects:
            if obj['name'] == robot_name:
                self.robot = obj
                break
        
        if self.robot is None:
            raise ValueError(f"Robot {robot_name} not found in simulation")
        
        self.sensor_sim = SimpleSensorSimulator(simulator)
        self.path = []
        self.path_index = 0
        
    def follow_path(self, path, threshold=0.5):
        """Follow a path using simple control"""
        if self.path_index >= len(path):
            print("Reached end of path")
            return True
        
        target = path[self.path_index]
        current_pos = (self.robot['x'] + self.robot['width']/2, 
                      self.robot['y'] + self.robot['height']/2)
        
        # Calculate direction to target
        dx = target[0] - current_pos[0]
        dy = target[1] - current_pos[1]
        distance = np.sqrt(dx**2 + dy**2)
        
        # Apply force proportional to distance
        force_scale = min(10.0, distance * 2)  # Limit max force
        fx = dx / distance * force_scale if distance > 0 else 0
        fy = dy / distance * force_scale if distance > 0 else 0
        
        # Apply the force
        self.simulator.apply_force(self.robot, fx, fy)
        
        # Check if we've reached the current waypoint
        if distance < threshold:
            self.path_index += 1
            print(f"Reached waypoint {self.path_index} of {len(path)}")
        
        return self.path_index >= len(path)
    
    def get_sensor_data(self):
        return {
            'lidar': self.sensor_sim.simulate_lidar(self.robot_name),
            'camera': self.sensor_sim.simulate_camera(self.robot_name)
        }

# Create a robot controller
controller = AdvancedRobotController(simulator, "Robot")
print("Robot controller initialized")

# Run the simulation with path following
controller.path = path
controller.path_index = 0

print("Starting path following simulation...")
for step in range(50):
    # Get sensor data
    sensor_data = controller.get_sensor_data()
    
    # Follow the path
    reached_goal = controller.follow_path(path)
    
    # Update simulation
    simulator.update(dt=0.1)
    
    if reached_goal:
        print(f"Goal reached at step {step + 1}!")
        break
    
    if step % 10 == 0:  # Print status every 10 steps
        robot_pos = (controller.robot['x'] + controller.robot['width']/2,
                    controller.robot['y'] + controller.robot['height']/2)
        print(f"Step {step + 1}: Robot at ({robot_pos[0]:.2f}, {robot_pos[1]:.2f})")

# Show final state
print(f"\nFinal robot position: ({controller.robot['x']:.2f}, {controller.robot['y']:.2f})")
simulator.render()

## Summary

In this notebook, we've covered:
1. Creating a simple physics simulation environment
2. Adding objects and robots to the simulation
3. Applying forces and simulating movement
4. Implementing sensor simulation (LiDAR and camera)
5. Basic path planning and navigation
6. Creating a robot controller for autonomous operation

Simulation environments are crucial for robotics development, allowing us to test algorithms safely before deployment on real hardware. The concepts learned here form the foundation for more advanced simulation platforms like Gazebo and Unity Robotics.

In [ ]:
# Clean up
print("\nChapter 2 complete! You've learned about simulation environments for robotics.")
print("Simulation concepts covered:")
print("- Physics simulation")
print("- Sensor simulation")
print("- Path planning")
print("- Robot control in simulation")